# F1 QA

## Ask questions using GPT4-turbo

## Import and definitions

In [61]:
from openai import OpenAI
from config import open_ai_key
from time import sleep
from itertools import cycle

In [60]:
client = OpenAI(api_key=open_ai_key)


def upload_file(file_path):
  # Upload a file with an "assistants" purpose
  upload_file = client.files.create(
  file=open(file_path, "rb"),
  purpose='assistants')

  return upload_file


def create_assistant(file):
  assistant = client.beta.assistants.create(
    instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}])

  # Add the file to the assistant
  assistant = client.beta.assistants.create(
    instructions="You are a mechanical designer chatbot. Use your knowledge base to best respond to customer queries.",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id])

  return assistant


def run_thread(question):
  thread = client.beta.threads.create()
  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question)
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,)
  status_complete = False
  for i in cycle(["|", "/", "-", "\\"]):
    print(f"Loading... {i}",end='\r') # '\r' clears the previous output
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id)
    status_complete = run.status == "completed"
    if status_complete:
      break
    sleep(0.5)

    # Retrieve the message object
    messages = client.beta.threads.messages.list(
      thread_id=thread.id)

  # Extract the message content
  message_content = messages.data[0].content[0].text
  annotations = message_content.annotations
  citations = []

  # Iterate over the annotations and add footnotes
  for index, annotation in enumerate(annotations):
      # Replace the text with a footnote
      message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

      # Gather citations based on annotation attributes
      if (file_citation := getattr(annotation, 'file_citation', None)):
          cited_file = client.files.retrieve(file_citation.file_id)
          citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
      elif (file_path := getattr(annotation, 'file_path', None)):
          cited_file = client.files.retrieve(file_path.file_id)
          citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
          # Note: File download functionality not implemented above for brevity

  # Add footnotes to the end of the message before displaying to user
  message_content.value += '\n\n' + '\n'.join(citations)

  return message_content.value

## Upload file and create new assistant

In [15]:
file_path = "texts/2022_formula_1_technical_regulations_-_iss_3_-_2021-02-19.pdf"

file = upload_file(file_path)
assistant = create_assistant(file)

## Ask questions

In [16]:
question = "What material can be used to manufacture the secondary roll structure?"

In [38]:
response = run_thread(question)
print(response)

The secondary roll structure, also known as the Halo, must be made to the standard FIA8869-2018 and must be supplied by an FIA designated manufacturer [0]. Therefore, individual teams do not manufacture this part themselves but must acquire it from a source that has been certified by the FIA to ensure compliance with safety and technical specifications.
[0] Secondary Roll Structure (Halo) 
The secondary roll structure which is not considered part of the survival cell must be
positioned symmetrically about the car centre plane with its front fixing axis at XC
= -975 and
Z=660. The mounting faces for the rearward fixings must lie on the plane Z=695. 
The secondary roll structure must be made to standard FIA8869-2018 and supplied by an FIA
designated manufacturer from 2022_formula_1_technical_regulations_-_iss_3_-_2021-02-19.pdf


In [47]:
question = "Can a fillet of 15mm be added to between the rear wing endplate and rear wing beam?"

In [48]:
response = run_thread(question)
print(response)

According to the 2022 Formula 1 Technical Regulations, for the rear wing endplate and rear wing beam assembly under Article 3.10.9:

"A fillet radius no greater than 10mm may be applied along the periphery of where these volumes intersect. Furthermore, once the trim is complete when viewed from the side no part of the Rear Wing Beam may be visible."

Therefore, based on the regulations, a fillet of 15mm between the rear wing endplate and rear wing beam would not be permissible as it exceeds the maximum allowed fillet radius of 10mm.



In [45]:
question = "What are all the components of the Rear Bodywork?"

In [46]:
response = run_thread(question)
print(response)

The components of the Rear Bodywork, as defined by the 2022 Formula 1 Technical Regulations, comprise the following:

1. **Coke Panel**: Must lie within a specific volume (RV-RBW-COKE) and be arranged to meet visibility restrictions when viewed from the side.

2. **Engine Cover**: Similarly, must lie within another predetermined volume (RV-RBW-EC) and also comply with visibility requisites from the side.

3. **Combination of Coke Panel and Engine Cover Surfaces**: The combination must form continuous curves with specified radii of curvature. They must not contain certain apertures or surfaces parallel to the X plane ahead of a specific X-coordinate (XR = -55).

4. **Apertures in the Bodywork**: Once defined, apertures can be added for suspension components, cooling flow exits, and auxiliary component cooling, within set size limits and placement conditions.

5. **Tail**: This area must fit within a designated volume (RV-TAIL) and also meets certain visibility restrictions.

6. **Exhaus

In [56]:
question = "How is the Ride Height affected by a change in the attitude of the unsprung mass?"

In [57]:
response = run_thread(question)
print(response)

The ride height, as outlined in the 2022 Formula 1 Technical Regulations, refers specifically to the "Legality ride height," which is the attitude of the unsprung mass in relation to the sprung mass for compliance with the technical regulations. The document mentions that at the legality ride height, with the wheels in a straight-ahead position, the origin of the wheels' coordinate system must be within a specific Z-coordinate range for both front and rear wheels [0].

While the document specifies the acceptable range for the legality ride height, it does not provide detailed information on how a change in the attitude of the unsprung mass directly affects the ride height. However, it can be inferred that the ride height would be influenced by any modifications to the suspension components or adjustments that alter the relative positions of the sprung and unsprung masses. Changes in the unsprung mass, such as variations in the stiffness, damping, or geometry of the suspension, can alte

In [51]:
question = "Generate a set of geometric constraint equations for the Coke Panel."

In [52]:
response = run_thread(question)
print(response)

To generate a set of geometric constraint equations for the Coke Panel as per the 2022 Formula 1 Technical Regulations, we need to consider the requirements stated in the document [0]:

1. The Coke Panel must lie within a predetermined volume (RV-RBW-COKE).
2. The Coke Panel must maintain a minimum distance of 50mm from RV-FLOOR-EDGE at every point.
3. From a side view, no part of RS-RBW-EC that lies within RV-RBW-COKE may be visible.

Additionally, for the combined external surfaces of the Coke Panel and Engine Cover:

- On any X section, the curve formed must be tangent continuous.
  - Outboard of Y=25, the radius of curvature must be at least 75mm if the curve is convex or 50mm if the curve is concave. However, within a specific rectangular region defined by \( XC = [20, 125] \) to \( XC = [150, 375] \), the radius of curvature must not be less than 25mm.
  - Between Y=5 and Y=25, the radius of curvature must be at least 25mm.

Based on these requirements, a set of geometric constra

In [58]:
question = "Which body component must maintain continuity with the Engine Cover?"

In [59]:
response = run_thread(question)
print(response)

The body component that must maintain continuity with the Engine Cover is the Sidepod Coke Panel. The external surfaces at the boundaries between these two parts must preserve both continuity and tangency in any X, Y, or Z plane [0].

[0] 3.7.7  Continuity
Once the Front and Rear Bodywork are fully defined the external surfaces at the boundaries
between adjacent sections of the Front Bodywork Sidepod Coke Panel and Engine Cover
parts must maintain continuity in any X Y or Z plane. 
Furthermore the external surfaces at the boundaries between adjacent sections of the
Sidepod Coke Panel and Engine Cover parts must also maintain tangency in any X Y or Z
plane from 2022_formula_1_technical_regulations_-_iss_3_-_2021-02-19.pdf
